In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
rank_df = pd.read_csv(filepath_or_buffer='/kaggle/input/dog-breeds/breed_rank.csv')
rank_df.head()

In [ ]:
traits_df = pd.read_csv(filepath_or_buffer='/kaggle/input/dog-breeds/breed_traits.csv')
traits_df.head()

In [ ]:
long_df = pd.read_csv(filepath_or_buffer='/kaggle/input/dog-breeds/breed_traits_long.csv')
long_df.head(n=10)

In [ ]:
long_numerical_df = long_df[~long_df['Trait'].isin({'Coat Type', 'Coat Length'})].copy()
long_numerical_df['score'] = long_numerical_df['Trait_Score'].astype(int) 
short_df = long_numerical_df[['Breed', 'score']].groupby(by='Breed').sum().reset_index()
short_df.shape

It may be dubious to assume that all of the scores are positive, in that 5 is more desirable than 1, and that we should weight them equally, but that's what we're going to do here.

In [ ]:
df = rank_df[['Breed', '2020 Rank']].merge(right=short_df, on='Breed', how='inner').merge(right=traits_df[['Breed', 'Coat Type', 'Coat Length']],
                                                                                         on='Breed', how='inner')
# here we want to convert the ranking, where lower is better, into something where larger is better
df['popularity'] = df['2020 Rank'].max() - df['2020 Rank']
df.head()

In [ ]:
from plotly.express import scatter
scatter(data_frame=df, x='score', y='popularity', hover_name='Breed', symbol='Coat Type', color='Coat Length',
       trendline='ols', trendline_scope='overall')

Our trendline is upward sloping, so we can very broadly say that objective measures as described are positively correlated with popularity, but our OLS has an r2 of about 0.1, so maybe it would be more accurate to say there's no accounting for taste.